In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
sys.path.append('./')

In [7]:
from decimal import Decimal
import math 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [8]:
from web_app.ekubo.main import EkuboAPIConnector, EkuboOrderBook
connector = EkuboAPIConnector()

TOKEN_A_DECIMALS = 18
TOKEN_B_DECIMALS = 6

token_a = "0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7"  # ETH
token_b = (
    "0x53c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8"  # USDC
)

### Calculating bids and asks for single selected pool

In [9]:
order_book = EkuboOrderBook(token_a, token_b)
pool_states = connector.get_pools()
df = pd.DataFrame(pool_states)
pool_df = df.loc[
    (df["token0"] == token_a) & (df["token1"] == token_b)
]

# Select one pool with specified fee, tick spacing (it should be the ETH/USDC pool with biggest TVL)

pool_df = pool_df[
    (pool_df['fee'].apply(lambda x: int(x, 0)) == 170141183460469235273462165868118016)
    &
    (pool_df['tick_spacing'].apply(lambda x: int(x)) == 1000)
]


In [10]:
row = pool_df.iloc[0]

In [11]:
liquidity_data = connector.get_pool_liquidity(row['key_hash'])
liquidity_data['data'] = sorted(liquidity_data['data'], key = lambda x: x['tick'])

In [12]:
# Get info about pool directly from the api
p = connector.get_pair_states(token_a, token_b)
test_pool = sorted(p['topPools'], key = lambda x: int(x['tvl0_total']))[::-1][0]
test_pool

{'fee': '170141183460469235273462165868118016',
 'tick_spacing': 1000,
 'extension': '0',
 'volume0_24h': '615590757058289023676',
 'volume1_24h': '2216702563776',
 'fees0_24h': '307795378529143882',
 'fees1_24h': '1108350481',
 'tvl0_total': '215463153193439084288',
 'tvl1_total': '1772100122148',
 'tvl0_delta_24h': '-32957724803352507484',
 'tvl1_delta_24h': '-200249167849'}

In [13]:
print(
    int(test_pool['tvl0_total']) / 10**18, 
    int(test_pool['tvl1_total']) / 10**6
)

215.4631531934391 1772100.122148


In [58]:
# Calculate asks
ask_ticks = [i for i in liquidity_data['data'] if i['tick']>=row['tick']]
asks = []
glob_liq = Decimal(row['liquidity'])

# Calculate for current tick (loops start with the next one)
next_tick = ask_ticks[0]['tick']
prev_tick = next_tick - row['tick_spacing'] 

prev_sqrt = order_book.get_sqrt_ratio(prev_tick) / 2**128
next_sqrt = order_book.get_sqrt_ratio(next_tick) / 2**128

supply = abs(((glob_liq / prev_sqrt) - (glob_liq / next_sqrt)) / 10**TOKEN_A_DECIMALS)
price = order_book.tick_to_price(prev_tick)
asks.append((price, supply))

for i, tick in enumerate(ask_ticks):
    if i == 0:
        continue
    glob_liq += Decimal(ask_ticks[i-1]['net_liquidity_delta_diff'])
    prev_tick = Decimal(ask_ticks[i-1]['tick'])
    curr_tick = Decimal(tick['tick'])
    
    prev_sqrt = order_book.get_sqrt_ratio(prev_tick) / 2**128
    next_sqrt = order_book.get_sqrt_ratio(curr_tick) / 2**128

    supply = abs(((glob_liq / prev_sqrt) - (glob_liq / next_sqrt)) / 10**TOKEN_A_DECIMALS)
    price = order_book.tick_to_price(prev_tick)
    asks.append((price, supply))
    

In [55]:
sum(i[1] for i in asks)

Decimal('208.105417705606739')

In [57]:
Decimal(test_pool['tvl0_total']) / 10**18

Decimal('215.463153193439084')

In [61]:
bid_ticks = [i for i in liquidity_data['data'] if i['tick']<=row['tick']][::-1]
bids = []
glob_liq = Decimal(row['liquidity'])

next_tick = bid_ticks[0]['tick']
prev_tick = next_tick + row['tick_spacing'] 

prev_sqrt = order_book.get_sqrt_ratio(prev_tick) / 2**128
next_sqrt = order_book.get_sqrt_ratio(next_tick) / 2**128

supply = abs(((glob_liq * prev_sqrt) - (glob_liq * next_sqrt)) / 10**TOKEN_B_DECIMALS)
price = order_book.tick_to_price(prev_tick)
bids.append((price, supply))

for i, tick in enumerate(bid_ticks):
    if i == 0:
        continue
    glob_liq -= Decimal(bid_ticks[i-1]['net_liquidity_delta_diff'])
    prev_tick = Decimal(bid_ticks[i-1]['tick'])
    curr_tick = Decimal(tick['tick'])
    
    prev_sqrt = order_book.get_sqrt_ratio(prev_tick) / 2**128
    next_sqrt = order_book.get_sqrt_ratio(curr_tick) / 2**128

    supply = abs(((glob_liq * prev_sqrt) - (glob_liq * next_sqrt))) / 10**TOKEN_B_DECIMALS
    price = order_book.tick_to_price(prev_tick)
    bids.append((price, supply))
    

In [63]:
sum(i[1] for i in bids) 

Decimal('1753886.78071443474')

In [64]:
Decimal(test_pool['tvl1_total']) / 10**6

Decimal('1772100.122148')